In [1]:
import pandas as pd
import numpy as np
import warnings
import os
warnings.filterwarnings('ignore')

In [2]:
print("\n1. LOADING RAW DATA FILES...")

# Load Census Data (Dataset 1)
census_raw = pd.read_excel('./source data/01 india-districts-census.xlsx')
print(f"Census data loaded: {census_raw.shape[0]} districts, {census_raw.shape[1]} columns")

# Load PMJDY Time Series (Dataset 2)
pmjdy_ts_raw = pd.read_excel('./source data/02 pmjdy_statewise_2020_2025.xlsx')
print(f"Time series data loaded: {pmjdy_ts_raw.shape[0]} states, {pmjdy_ts_raw.shape[1]} columns")

# Load Current PMJDY Report (Dataset 3)
pmjdy_curr_raw = pd.read_excel('./source data/03 Statewise account opening Report as on 09 07 2025.xlsx')
print(f"Current report loaded: {pmjdy_curr_raw.shape[0]} rows, {pmjdy_curr_raw.shape[1]} columns")


1. LOADING RAW DATA FILES...
Census data loaded: 640 districts, 118 columns
Time series data loaded: 36 states, 14 columns
Current report loaded: 37 rows, 7 columns


In [3]:
print("\n2. CLEANING TIME SERIES DATA (2020-2025)...")

# Create clean dataframe
pmjdy_ts_clean = pmjdy_ts_raw.copy()

# Fix state names
pmjdy_ts_clean['State/UT'] = pmjdy_ts_clean['State/UT'].replace('Islands', 'Andaman And Nicobar Islands')
print("Standardized state names")

# Convert from lakhs to actual numbers
value_columns = ['Mar20_Total', 'Mar20_Operative', 'Mar21_Total', 'Mar21_Operative',
                 'Mar22_Total', 'Mar22_Operative', 'Mar23_Total', 'Mar23_Operative',
                 'Mar24_Total', 'Mar24_Operative', 'Jan25_Total', 'Jan25_Operative']

for col in value_columns:
    pmjdy_ts_clean[col] = (pmjdy_ts_clean[col] * 100000).round(0).astype(int)
print("Converted values from lakhs to actual numbers")

# Calculate operative rates for each period
periods = ['Mar20', 'Mar21', 'Mar22', 'Mar23', 'Mar24', 'Jan25']
for period in periods:
    pmjdy_ts_clean[f'{period}_Op_Rate'] = (
        pmjdy_ts_clean[f'{period}_Operative'] / pmjdy_ts_clean[f'{period}_Total'] * 100
    ).round(2)
print("Calculated operative rates for all periods")

# Calculate growth metrics
pmjdy_ts_clean['CAGR_2020_25'] = (np.power(pmjdy_ts_clean['Jan25_Total'] / pmjdy_ts_clean['Mar20_Total'], 1/4.83) - 1) * 100
pmjdy_ts_clean['CAGR_2020_25'] = pmjdy_ts_clean['CAGR_2020_25'].round(2)

pmjdy_ts_clean['Growth_2024_25'] = (
    (pmjdy_ts_clean['Jan25_Total'] - pmjdy_ts_clean['Mar24_Total']) / 
    pmjdy_ts_clean['Mar24_Total'] * 100
).round(2)

# Year-over-year growth rates for ML models
for i, (year1, year2) in enumerate(zip(periods[:-1], periods[1:])):
    pmjdy_ts_clean[f'Growth_{year1}_{year2}'] = (
        (pmjdy_ts_clean[f'{year2}_Total'] - pmjdy_ts_clean[f'{year1}_Total']) / 
        pmjdy_ts_clean[f'{year1}_Total'] * 100
    ).round(2)

print("Calculated growth metrics (CAGR, YoY growth)")
print(f"Final shape: {pmjdy_ts_clean.shape}")


2. CLEANING TIME SERIES DATA (2020-2025)...
Standardized state names
Converted values from lakhs to actual numbers
Calculated operative rates for all periods
Calculated growth metrics (CAGR, YoY growth)
Final shape: (36, 27)


In [4]:
print("\n3. CLEANING CURRENT REPORT (JULY 2025)...")

# Remove total row
pmjdy_curr_clean = pmjdy_curr_raw[pmjdy_curr_raw['State Name'] != 'Total'].copy()
print(f"Removed 'Total' row, {len(pmjdy_curr_clean)} states remain")

# Rename columns for clarity
pmjdy_curr_clean.columns = ['S.No', 'State_Name', 'Rural_Beneficiaries', 
                            'Urban_Beneficiaries', 'Total_Beneficiaries',
                            'Balance_Crores', 'RuPay_Cards']

# Convert balance to rupees
pmjdy_curr_clean['Balance_Rupees'] = pmjdy_curr_clean['Balance_Crores'] * 10000000
print("Converted balance from crores to rupees")

# Calculate derived metrics
pmjdy_curr_clean['Rural_Percent'] = (
    pmjdy_curr_clean['Rural_Beneficiaries'] / pmjdy_curr_clean['Total_Beneficiaries'] * 100
).round(2)

pmjdy_curr_clean['Urban_Percent'] = (
    pmjdy_curr_clean['Urban_Beneficiaries'] / pmjdy_curr_clean['Total_Beneficiaries'] * 100
).round(2)

pmjdy_curr_clean['RuPay_Penetration'] = (
    pmjdy_curr_clean['RuPay_Cards'] / pmjdy_curr_clean['Total_Beneficiaries'] * 100
).round(2)

pmjdy_curr_clean['Avg_Balance_Rs'] = (
    pmjdy_curr_clean['Balance_Rupees'] / pmjdy_curr_clean['Total_Beneficiaries']
).round(2)

pmjdy_curr_clean['Rural_Urban_Ratio'] = (
    pmjdy_curr_clean['Rural_Beneficiaries'] / pmjdy_curr_clean['Urban_Beneficiaries']
).round(3)

print("Calculated rural/urban percentages")
print("Calculated RuPay penetration rate")
print("Calculated average balance per account")
print(f"Final shape: {pmjdy_curr_clean.shape}")


3. CLEANING CURRENT REPORT (JULY 2025)...
Removed 'Total' row, 36 states remain
Converted balance from crores to rupees
Calculated rural/urban percentages
Calculated RuPay penetration rate
Calculated average balance per account
Final shape: (36, 13)


In [5]:
print("\n4. AGGREGATING CENSUS DATA TO STATE LEVEL...")

# Key census columns to aggregate
census_agg = census_raw.groupby('State name').agg({
    'District code': 'count',  # Count of districts
    'Population': 'sum',
    'Households': 'sum',
    'Rural_Households': 'sum',
    'Urban_Households': 'sum',
    'Literate': 'sum',
    'Workers': 'sum',
    'Households_with_Internet': 'sum',
    'Households_with_Computer': 'sum',
    'Households_with_Telephone_Mobile_Phone': 'sum',
    'Housholds_with_Electric_Lighting': 'sum',
    'LPG_or_PNG_Households': 'sum',
    'Male': 'sum',
    'Female': 'sum'
}).reset_index()

# Rename columns
census_agg.rename(columns={
    'State name': 'State_Name',
    'District code': 'Total_Districts'
}, inplace=True)

# Calculate percentages and ratios
census_agg['Rural_HH_Percent'] = (census_agg['Rural_Households'] / census_agg['Households'] * 100).round(2)
census_agg['Urban_HH_Percent'] = (census_agg['Urban_Households'] / census_agg['Households'] * 100).round(2)
census_agg['Literacy_Rate'] = (census_agg['Literate'] / census_agg['Population'] * 100).round(2)
census_agg['Internet_Penetration'] = (census_agg['Households_with_Internet'] / census_agg['Households'] * 100).round(2)
census_agg['Phone_Penetration'] = (census_agg['Households_with_Telephone_Mobile_Phone'] / census_agg['Households'] * 100).round(2)
census_agg['Electricity_Access'] = (census_agg['Housholds_with_Electric_Lighting'] / census_agg['Households'] * 100).round(2)
census_agg['Gender_Ratio'] = (census_agg['Female'] / census_agg['Male'] * 1000).round(0)

print(f"Aggregated {len(census_raw)} districts to {len(census_agg)} states")
print("Calculated literacy rate, rural/urban percentages")
print("Calculated infrastructure penetration metrics")
print(f"Final shape: {census_agg.shape}")


4. AGGREGATING CENSUS DATA TO STATE LEVEL...
Aggregated 640 districts to 35 states
Calculated literacy rate, rural/urban percentages
Calculated infrastructure penetration metrics
Final shape: (35, 22)


In [6]:
print("\n5. CREATING INTEGRATED ML DATASET...")

# Standardize state names for merging
pmjdy_ts_clean['State_Name_Std'] = pmjdy_ts_clean['State/UT'].str.upper().str.strip()
pmjdy_curr_clean['State_Name_Std'] = pmjdy_curr_clean['State_Name'].str.upper().str.strip()
census_agg['State_Name_Std'] = census_agg['State_Name'].str.upper().str.strip()

# Merge time series with current data
ml_dataset = pmjdy_ts_clean.merge(
    pmjdy_curr_clean[['State_Name_Std', 'Rural_Beneficiaries', 'Urban_Beneficiaries',
                      'Total_Beneficiaries', 'Balance_Rupees', 'RuPay_Cards',
                      'Rural_Percent', 'RuPay_Penetration', 'Avg_Balance_Rs',
                      'Rural_Urban_Ratio']],
    on='State_Name_Std',
    how='left'
)

# Merge with census data
ml_dataset = ml_dataset.merge(
    census_agg[['State_Name_Std', 'Population', 'Households', 'Literacy_Rate',
                'Rural_HH_Percent', 'Internet_Penetration', 'Phone_Penetration']],
    on='State_Name_Std',
    how='left'
)

# Create ML features as per methodology
ml_dataset['Account_Density_Per_Lakh'] = (
    ml_dataset['Total_Beneficiaries'] / (ml_dataset['Population'] / 100000)
).round(2)

ml_dataset['High_Operative_Flag'] = (ml_dataset['Jan25_Op_Rate'] > 75).astype(int)
ml_dataset['High_Growth_Flag'] = (ml_dataset['Growth_2024_25'] > 40).astype(int)
ml_dataset['High_RuPay_Flag'] = (ml_dataset['RuPay_Penetration'] > 70).astype(int)
ml_dataset['High_Balance_Flag'] = (ml_dataset['Avg_Balance_Rs'] > 4000).astype(int)
ml_dataset['Rural_Dominated_Flag'] = (ml_dataset['Rural_Percent'] > 70).astype(int)

# Calculate operative trend features
operative_cols = [f'{period}_Op_Rate' for period in periods]
ml_dataset['Operative_Mean'] = ml_dataset[operative_cols].mean(axis=1).round(2)
ml_dataset['Operative_Std'] = ml_dataset[operative_cols].std(axis=1).round(2)
ml_dataset['Operative_Trend'] = ml_dataset[operative_cols].apply(
    lambda x: np.polyfit(range(len(x)), x, 1)[0], axis=1
).round(3)

print(f"Successfully integrated data for {len(ml_dataset)} states")
print("Created binary classification flags")
print("Calculated density and trend metrics")
print(f"Final ML dataset shape: {ml_dataset.shape}")


5. CREATING INTEGRATED ML DATASET...
Successfully integrated data for 36 states
Created binary classification flags
Calculated density and trend metrics
Final ML dataset shape: (36, 52)


In [7]:
print("\n6. SAVING CLEANED DATASETS...")

# Create folder if it doesn't exist
os.makedirs("cleaned datasets", exist_ok=True)

# Save individual cleaned datasets
pmjdy_ts_clean.to_csv('./cleaned datasets/cleaned_pmjdy_timeseries.csv', index=False)
print("Saved: cleaned_pmjdy_timeseries.csv")

pmjdy_curr_clean.to_csv('./cleaned datasets/cleaned_pmjdy_current.csv', index=False)
print("Saved: cleaned_pmjdy_current.csv")

census_agg.to_csv('./cleaned datasets/cleaned_census_statewise.csv', index=False)
print("Saved: cleaned_census_statewise.csv")

# Save integrated ML dataset
ml_dataset.to_csv('./cleaned datasets/ml_integrated_dataset.csv', index=False)
print("Saved: ml_integrated_dataset.csv")

# Save to Excel with multiple sheets
with pd.ExcelWriter('./cleaned datasets/pmjdy_cleaned_all.xlsx', engine='openpyxl') as writer:
    pmjdy_ts_clean.to_excel(writer, sheet_name='Time_Series', index=False)
    pmjdy_curr_clean.to_excel(writer, sheet_name='Current_Report', index=False)
    census_agg.to_excel(writer, sheet_name='Census_State', index=False)
    ml_dataset.to_excel(writer, sheet_name='ML_Dataset', index=False)
print("Saved: pmjdy_cleaned_all.xlsx (all sheets)")


6. SAVING CLEANED DATASETS...
Saved: cleaned_pmjdy_timeseries.csv
Saved: cleaned_pmjdy_current.csv
Saved: cleaned_census_statewise.csv
Saved: ml_integrated_dataset.csv
Saved: pmjdy_cleaned_all.xlsx (all sheets)


In [8]:
print("DATA CLEANING SUMMARY")
print("\nKey Statistics:")
print(f"Total states/UTs processed: {len(ml_dataset)}")
print(f"States with >75% operative rate: {ml_dataset['High_Operative_Flag'].sum()}")
print(f"States with >40% recent growth: {ml_dataset['High_Growth_Flag'].sum()}")
print(f"States with >70% RuPay penetration: {ml_dataset['High_RuPay_Flag'].sum()}")
print(f"Rural-dominated states (>70% rural): {ml_dataset['Rural_Dominated_Flag'].sum()}")

print("\nData Quality Checks:")
print(f"Missing population data: {ml_dataset['Population'].isna().sum()} states")
print(f"Data completeness: {(1 - ml_dataset.isna().sum().sum() / ml_dataset.size) * 100:.1f}%")

print("\nDATA CLEANING COMPLETED SUCCESSFULLY!")

DATA CLEANING SUMMARY

Key Statistics:
Total states/UTs processed: 36
States with >75% operative rate: 25
States with >40% recent growth: 0
States with >70% RuPay penetration: 14
Rural-dominated states (>70% rural): 14

Data Quality Checks:
Missing population data: 7 states
Data completeness: 96.4%

DATA CLEANING COMPLETED SUCCESSFULLY!
